## Home work 6

In [12]:
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from typing import Tuple
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from mlflow.models import infer_signature
import mlflow

In [13]:
import pickle


### Loading Kedro Extension

In this section, we load the Kedro extension for IPython. Kedro is a development workflow framework that helps in creating reproducible, maintainable, and modular data science code. By loading this extension, we can leverage Kedro's capabilities within our Jupyter Notebook environment.



In [14]:
%load_ext kedro.ipython


SyntaxError: unterminated string literal (detected at line 5) (2768163962.py, line 5)

### Using Kedro Catalog

After loading the Kedro extension with `%load_ext kedro.ipython`, you can interact with the Kedro catalog directly in your Jupyter Notebook. The `catalog` object provides access to the data and model artifacts defined in your Kedro project.

For example, calling `catalog` will display a list of all available datasets and models that you can load and use in your notebook. This is useful for quickly inspecting what resources are available in your Kedro pipeline.

To load a specific dataset or model, you can use the `catalog.load("name_of_dataset_or_model")` method. This allows you to seamlessly integrate Kedro's data management capabilities into your interactive data science workflow.

In [4]:
catalog


{'dataset_id_214': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/01_raw/dataset_id_214.csv'), "
                   "protocol='file', load_args={}, save_args={'index': False})",
 'inference_data': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/01_raw/sampled_file.csv'), "
                   "protocol='file', load_args={}, save_args={'index': False})",
 'encoder': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/03_primary/encoder.pkl'), "
            "backend='pickle', protocol='file', load_args={}, save_args={})",
 'preprocessed_data': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/02_modelinput/preprocessed_data.csv'), "
                      "protocol='file', load_args={}, save_a

In [15]:
def preprocess_data(df: pd.DataFrame,encoder:OneHotEncoder) -> Tuple[pd.DataFrame]:
    """
    Preprocesses the input DataFrame by converting categorical columns into one-hot encoded columns
    using sklearn's OneHotEncoder. The numerical columns are retained and concatenated with the
    encoded categorical columns.

    Parameters:
    -----------
    df : pd.DataFrame
        The input DataFrame containing both categorical and numerical columns.
    encoder: OneHotEncoder
        The one hot encoder that has been stored in the configuration
    Returns:
    --------
    Tuple[pd.DataFrame, OneHotEncoder]
        - A DataFrame with categorical columns one-hot encoded and numerical columns retained.
        # - The fitted OneHotEncoder instance.
    """
    # Identify numeric and categorical columns
    num_cols = df._get_numeric_data()
    cat_cols = df.select_dtypes(include=['object'])
    
    if not cat_cols.empty:
        # Initialize and fit the OneHotEncoder on categorical columns
        # encoder = OneHotEncoder(sparse_output=False, dtype='float', drop=None)
        cat_cols_ohe = encoder.fit_transform(cat_cols)
        
        # Convert the encoded columns into a DataFrame
        cat_cols_ohe_df = pd.DataFrame(
            cat_cols_ohe, columns=encoder.get_feature_names_out(cat_cols.columns)
        )
        
        # Concatenate the one-hot encoded categorical columns with the numeric columns
        df_processed = pd.concat([cat_cols_ohe_df, num_cols.reset_index(drop=True)], axis=1)
    else:
        df_processed = df
        encoder = None

    # Save the processed DataFrame to a CSV file
    # df_processed.to_csv("C:\\Users\\Admin\\Desktop\\AI-839\\srinivasan-ai-839\\data\\01_raw\\pre_processed.csv", index=False)
    
    return df_processed

### Encoder

Retrieving the encoder stored in the catalog as a pickle file.


In [3]:
encoder = catalog.load("encoder")

NameError: name 'catalog' is not defined

In [17]:
dataset = catalog.load("preprocessed_data")

[10/16/24 22:02:39] INFO     Loading data from preprocessed_data (CSVDataset)...                ]8;id=315762;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=383097;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

loading the model for the same.

In [18]:
model = catalog.load("model2")

[10/16/24 22:02:41] INFO     Loading data from model2 (PickleDataset)...                        ]8;id=234422;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=837279;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [19]:
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [27]:
dataset_new = preprocess_data(dataset,encoder)

removing the first ten rows from the dataset.


In [28]:
dataset_truncate = dataset_new[10:]

In [29]:
dataset_truncate.drop(columns=["Unnamed: 0"],inplace=True)
dataset_truncate


,checking_status_0<=X<200,checking_status_<0,checking_status_>=200,checking_status_no checking,credit_history_all paid,credit_history_critical/other existing credit,credit_history_delayed previously,credit_history_existing paid,credit_history_no credits/all paid,purpose_business,...,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.472068,0.419630,0.652240,0.945691,0.115385,0.871547,0.927647,0.664855,0.014704,True
11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.424463,0.240885,0.958707,0.691337,0.691825,0.732606,0.748178,0.923704,0.786498,True
12,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.317776,0.400655,0.610133,0.528451,0.184162,0.786356,0.064903,0.429595,0.005475,False
13,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.552385,0.799394,0.859617,0.199796,0.091506,0.215622,0.830488,0.448498,0.492287,True
14,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.917147,0.128064,0.727007,0.199042,0.611148,0.545832,0.234905,0.553738,0.847184,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.882218,0.883794,0.233441,0.150932,0.499922,0.434244,0.140232,0.187078,0.141813,False
96,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.149197,0.043498,0.134882,0.350446,0.889813,0.760545,0.261638,0.823732,0.053394,True
97,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.424718,0.253165,0.242619,0.903971,0.694778,0.517654,0.707042,0.575040,0.073795,False
98,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.469677,0.654652,0.316479,0.831883,0.405854,0.259628,0.350143,0.868693,0.065413,True


In [30]:
X = dataset_truncate.drop(columns=["y"])

In [32]:
Y = dataset_truncate['y']

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42
    )

training the model again.

In [34]:
# model = RandomForestClassifier(random_state=42)
model.fit(X_train,Y_train)

RandomForestClassifier(random_state=42)

running the inference with the new model.

In [35]:
Y_pred = model.predict(X_test)

In [36]:
accuracy_score(Y_test,Y_pred)

0.9444444444444444

saving the new model

In [39]:
import pickle
with open("model_new.pkl", "wb") as f: 
    pickle.dump(model, f)

# Implementing Right to Erasure with Re-scoring of Predictions

To ensure compliance with the "right to erasure," including re-scoring of past predictions influenced by the records being erased, follow the steps below:

## 1. Data Management and Traceability

- **Data Versioning:**  
  Implement a data versioning system, such as `DVC` or `MLflow`, to track changes in datasets. Ensure each record has a unique identifier, e.g., UUID.

- **Influence Tracking:**  
  Record the data used for training and predictions with a **data lineage system**. Track which records influenced particular model predictions.

## 2. Identify Affected Models

- **Model Versioning:**  
  Use tools like `MLflow` or `Weights and Biases` to version models and track the datasets they were trained on.

- **Find Influenced Models:**  
  Query the version control system to identify models trained on datasets containing the records marked for erasure.

## 3. Retrain Models

- **Remove the 10 Records:**  
  Create a new dataset version that excludes the 10 records. Use the data versioning tool to do this.

- **Retraining Options:**
  - **Full Retraining:**  
    Retrain the model from scratch using the updated dataset to ensure no influence from the erased records.
  - **Incremental Training:**  
    If full retraining is computationally expensive, use **incremental learning** to adjust the model based on the changes in the dataset.

## 4. Re-score Past Predictions

- **Backlog of Predictions:**  
  Store all past predictions, along with references to the data and models used, in a **prediction database**.

- **Re-scoring Process:**
  - **Identify Affected Predictions:**  
    Use the data lineage system to identify which past predictions were influenced by models trained on datasets containing the 10 records.
  - **Re-score Affected Predictions:**  
    Once the model has been retrained, re-run the past predictions through the updated model to generate new predictions.
  - **Update Databases:**  
    Replace the old predictions in the database with the new predictions and log this process for auditing.

## 5. Communicating the Changes

- **Auditing:**  
  Maintain logs of the data erasure, model retraining, and prediction updates for transparency.
  
- **Notifying Users:**  
  Inform users if their data was affected by the "right to erasure" and how the system has been updated accordingly.
